In [1]:
#2-step keyword extraction
#Text Mining Domains, VU

import pickle
import glob
import pandas as pd
import json
import os

def new_dict_concat_six (unpickled_dict):
    '''
    takes an unpickled dictionary as input with headline as [1] and list of first five sentences as [2]
    converts to a new dict with same key but value is string of concatenated headline + first five sentences
    filters the values of the keywords
    reurns: new dictionary with keys and concatenated headlines and first_five sentences of those articles which 
        contain the keywords
    '''        
    output = [unpickled_dict[key][1]+ '. ' + ' '.join(unpickled_dict[key][2]) for key in unpickled_dict]
    
    # calling test_dict only gets keys
    new_dict = dict(zip(unpickled_dict, output))
    
    return new_dict

def first_filter(keywords, infolder):
    for file in glob.glob(infolder):
        #prep filepaths for writing files
        path = os.path.split(file)[0].split('/')[0]
        basename = os.path.basename(file)

        unpickled_dict = pickle.load(open (file, 'rb'))
        #print(unpickled_dict)
        # run the function above to concat headlines and first_five
        new_dict = new_dict_concat_six(unpickled_dict)
        #print(new_dict)
        #print(new_dict)
        #rough filter for only keywords (see above)
        output_dict = {k: v for k, v in new_dict.items() for keyword in keywords if keyword in v}

        #write filtered articles - choosing json this time so I can inspect them in IDE
        if not os.path.exists(f'filteredpickles/{keywords[0]}'):
            os.makedirs(f'filteredpickles/{keywords[0]}')

        with open(f'filteredpickles/{keywords[0]}/{basename.rstrip(".gz.pkl")}.json', 'w', encoding = 'utf-8') as outfile:
            json.dump(output_dict, outfile)
                  
def keyword_search(keywords, big_df, treshold = 0):
#phase 2 of keyword search
    articles = []
    article_keywords = []
    article_ids = []
    #do a keyword search for the company
    i = 0
    for text in big_df['text']:
        keywords_found = set()
        for keyword in keywords:
            if keyword in text:
                keywords_found.add(keyword)
        if len(keywords_found) >= treshold:
            articles.append(text)
            article_keywords.append(keywords_found)
            article_ids.append(big_df.index[i])
        i += 1
    return articles, article_ids, article_keywords
                  
def keyword_search2(keywords, big_df, treshold = 0):
#phase 2 of keyword search
    article_dictlist = []
    #do a keyword search for the company
    i = 0
    for text in big_df['text']:
        keywords_found = set()
        for keyword in keywords:
            if keyword in text:
                keywords_found.add(keyword)
        if len(keywords_found) >= treshold:
            article_dict = {'Identifier' : big_df.index[i], 'Text' : text, 'Keywords' : keywords_found, 'Number' : i}
            article_dictlist.append(article_dict)
        i += 1
    articles = pd.DataFrame(article_dictlist)
    return articles
                  
def first_filter2(keywords, infolder):
    for file in glob.glob(infolder):
        #prep filepaths for writing files
        path = os.path.split(file)[0].split('/')[0]
        basename = os.path.basename(file)

        unpickled_dict = pickle.load(open (file, 'rb'))
        #print(unpickled_dict)
        # run the function above to concat headlines and first_five
        new_dict = new_dict_concat_six(unpickled_dict)
        #print(new_dict)
        #print(new_dict)
        #rough filter for only keywords (see above)
        output_dict = {k: v for k, v in new_dict.items() for keyword in keywords if keyword in v}

        #write filtered articles - choosing json this time so I can inspect them in IDE
        if not os.path.exists(f'filteredpickles/{keywords[0]}'):
            os.makedirs(f'filteredpickles/{keywords[0]}')

        with open(f'filteredpickles/{keywords[0]}/{basename.rstrip(".gz.pkl")}.json', 'w', encoding = 'utf-8') as outfile:
            json.dump(output_dict, outfile)

In [2]:
#define the keywords for the 1st rough search
keywords=['poverty', 'Poverty']

#rough filter, choose articles containing 'poverty' or 'aid',
#write them to json files
#computationally the most heavy step

infolder = 'pickles/*'
#create json files for all subcorpora with 1 or more docs containing the keyword(s).
first_filter(keywords, infolder)

In [3]:
keyword = keywords[0]
#read in the filtered json files
df_list = []
for file in glob.glob(f'filteredpickles/{keyword}/*'):
    jsonners = json.load(open(file, 'rb'))
    columns=['text']
    df_list.append(pd.DataFrame.from_dict(jsonners, orient='index', columns= columns))
big_df = pd.concat(df_list)
pd.set_option('display.max_colwidth', None)
print(f'The number of articles left after the first filter is: {len(big_df)}')

The number of articles left after the first filter is: 54455


In [4]:
topic = 'poverty'

with open(f'keywords/{topic}.txt', 'r', encoding = 'utf-8') as infile:
    keywords2 = infile.read().splitlines()
    
articles = keyword_search2(keywords2, big_df, treshold = 3)
#print(articles)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))

#to do: figure out how to apply vectorization to the entire corpus
#or: select a representative example to train the tfidfvectorizer!

#2nd to do: find a way to give tf-idf scores to 2-grams and 3-grams

#train and apply Tfidf
vectorizer = TfidfVectorizer(use_idf = True, stop_words = stopWords)
tf_idf_vecs = vectorizer.fit_transform(big_df['text'])

In [6]:
keywords_corpus = []
for keyword in keywords2:
    if keyword in vectorizer.get_feature_names():
        keywords_corpus.append(keyword)
print('keywords used:', keywords_corpus)
            

keywords used: ['vulnerable', 'poor', 'ownership', 'control', 'property', 'resilience', 'exposure', 'shocks', 'disasters', 'mobilization', 'inhuman', 'workers', 'extortion', 'slavery', 'famine', 'hunger', 'entrepreneur', 'development']


In [7]:
#print(tf_idf_vecs)
#third step:
#compute tf-idf scores for selected docs. takes a few minutes
doc_scores = []
for i in articles['Number']:
    df = pd.DataFrame(tf_idf_vecs[i].T.todense(), index=vectorizer.get_feature_names(), columns=["tfidf"])
    df = df.sort_values(by=["tfidf"], ascending=False)
    doc_score = 0
    keywords_found = set()
    for keyword in keywords_corpus:
        keyword_score = df['tfidf'][keyword]
        if keyword_score > 0:
            keywords_found.add(keyword)
            doc_score += keyword_score
    doc_scores.append(doc_score)


In [8]:
scores_df = pd.DataFrame(doc_scores, columns = ['Score'])

In [9]:
articles_w_scores = pd.concat([articles, scores_df], axis = 1)

In [10]:
articles_complete = articles_w_scores.drop('Number', axis=1)

In [11]:
articles_complete = articles_complete.sort_values(by="Score", ascending = False)

In [12]:
articles_complete[:50]

,Identifier,Text,Keywords,Score
442,APW_ENG_20051120.0537,"Britain pledges money to international fund to help poor countries deal with economic shocks. Britain will contribute 50 million pounds (US$85 million; euro75 million) to a new international fund that helps poor countries deal with economic shocks, Treasury chief Gordon Brown announced Sunday. The International Monetary Fund scheme will offer relief in the wake of high oil prices, natural disasters, conflicts or other adverse developments. The fund is expected to be capable of subsidizing lending of more than 1.5 billion pounds (US$2.8 billion; euro2.3 billion). Brown said global action is needed in response to the doubling of oil prices over the last year _ ""the largest sustained oil price shock since the 1970s."" ""This new IMF facility will play a vital role supporting poverty reduction in some of the world's most vulnerable countries, ensuring that development is not undermined by changes in economic conditions, whether those come from terms of trade shocks, high oil prices, or natural disasters,"" he said.","{development, vulnerable, disasters, shocks, poor, natural disaster}",0.751235
436,APW_ENG_20050915.1138,"Venezuela expands land reform, claiming farmlands for poor and stirring some concerns. Venezuela has claimed about 600,000 hectares (about 1.5 million acres) of farmlands as state property to be turned over to poor farmers, giving hope to some while raising concern among others about the future of private property in the country. Vice President Jose Vicente Rangel defended the agrarian reform program Thursday, saying Venezuela ""is broadening the system of private property"" by making lands available to poor farmers. ""That is social justice,"" he said. The government's National Lands Institute so far has declared 21 ranches to be state property, saying they were determined to be ""idle"" and those who claimed ownership couldn't prove it through documents. The government has helped poor farmers form cooperatives to start to work the land, despite legal challenges from those who used to run the ranches. President Hugo Chavez, who on Thursday took his anti-poverty message to the United Nations, said earlier this week that ""private property has to be subordinate to the general interest."" However, he has insisted private property rights will be upheld, as outlined in Venezuela's constitution.","{ownership, property, poor}",0.704637
450,APW_ENG_20060222.0356,"Norway contributes to new IMF fund to help poor countries handle economic shocks. Norway pledged 240 million kroner (US$35.5 million or euro30 million) on Wednesday to a new global fund to help poor countries deal with economic shocks. The program was set up by the International Monetary Fund in November to provide highly subsidized loans to poor countries when their economies are hit by war, natural disaster or extreme fluctuations in import or export prices. ""This new scheme will act as a buffer and will help to ensure continuity in the fight against poverty, even in cases where vulnerable, poor countries have to deal with sudden severe economic shocks due to external circumstances,"" Norwegian Aid Minister Erik Solheim said. The fund, called the Exogenous Shocks Facility, was set up as part of the IMF's Poverty Reduction and Growth Facility Trust following a proposal from the world's richest nations, called the Group of Eight. Norway, which is rich on offshore oil, is the world's largest foreign aid donor per capita.","{natural disaster, shocks, vulnerable, poor}",0.683885
1701,XIN_ENG_20091202.0200,"UN chief calls for eradication of all slavery practices on\ninternational day of slavery abolition. Contemporary forms of slavery remain a ""grave and unresolved problem"" across all continents, UN Secretary-general Ban Ki-moon warned on Wednesday, calling for greater efforts to address poverty and social inequalities which leave people vulnerable to enslavement. In a message marking the International D

In [20]:
file1 = 'pickles/afp_eng_199501.gz.pkl'
keywords=['poverty', 'Poverty']
unpickled = pickle.load(open(file1, 'rb'))

new_dict = new_dict_concat_six(unpickled)
output_dict = {k: v for k, v in new_dict.items() for keyword in keywords if keyword in v}
print(output_dict)

{'AFP_ENG_19950102.0301': "Budget deficit weighs on new Brazilian president's vow to fight poverty. Brazil's new president Fernando Henrique Cardoso has promised to fight poverty with ambitious economic reforms but must first conquer the government budget deficit, experts say. When Cardoso took office Sunday, the 1995 government budget included a deficit of more than 10 billion dollars, equal to almost two percent of the country's gross domestic product. To narrow the budget gap, Cardoso will have to fend off enormous pressure to spend more on social welfare and health programs, experts say. That balancing act will be precarious, they say, because Cardoso has vowed to help the poor masses, even if it means the powerful rich minority will suffer. Cardoso hopes his plans will be boosted by a free trade pact with Argentina, Uruguay and Paraguay that went into effect on Sunday. Mercosur, as the pact is known, combines 200 million consumers with a gross domestic product of 800 billion dolla

In [ ]:
#function to find the articles containing the company name and keywords
def keyword_company_search(company, keywords, big_df, treshold = 0):
    articles = []
    article_keywords = []
    
    found_company = False
    for text in big_df['text']:
        if company in text:   
            keywords_found = set()
            for keyword in keywords:
                if keyword in text:
                    keywords_found.add(keyword)
            if len(keywords_found) >= treshold:
                articles.append(text)
                article_keywords.append(keywords_found)
    return articles, article_keywords

with open('keywords/gender equality.txt', 'r', encoding = 'utf-8') as infile:
    keywordz = infile.read().splitlines()

articles, keywords_per_article = keyword_company_search('', keywordz, big_df, treshold = 3)

print(len(articles))